In [1]:
!pip install tensorflow
!pip install autokeras
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 458.3 MB 15 kB/s               ��█████████████████████████  | 428.5 MB 54.8 MB/s eta 0:00:01 
     |████████████████████████████████| 5.6 MB 81.1 MB/s            
     |████████████████████████████████| 57 kB 8.4 MB/s             
     |████████████████████████████████| 462 kB 73.5 MB/s            
     |████████████████████████████████| 42 kB 1.9 MB/s             
     |████████████████████████████████| 65 kB 6.1 MB/s             
     |████████████████████████████████| 1.3 MB 67.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.0 MB 59.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 781 kB 57.6 MB/s            
     |████████████████████████████████| 4.9 MB 50.8 MB/s            
     |████████████████████████████████| 97 kB 9.4 MB/s             
     |██████████████

In [1]:
def download_kaggle_dataset(kaggle_user:str, kaggle_key:str, data_set:str, path:str):
    import os
    import glob
    os.environ['KAGGLE_USERNAME'] = kaggle_user 
    os.environ['KAGGLE_KEY'] = kaggle_key

    import kaggle
    from kaggle.api.kaggle_api_extended import KaggleApi
    os.chdir(os.getcwd())
    os.system("mkdir " + path)
    os.chdir(path)
    api = KaggleApi()
    api.authenticate()
    api.competition_download_file('facial-keypoints-detection','training.zip')
    api.competition_download_file('facial-keypoints-detection','test.zip')


download_kaggle_dataset("username", "key", "facial-keypoints-detection","my_data")



100%|██████████| 60.1M/60.1M [00:00<00:00, 241MB/s]


100%|██████████| 16.0M/16.0M [00:00<00:00, 130MB/s] 

In [5]:
import numpy as np
import os
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import tensorflow as tf                
import pandas as pd
from tensorflow.keras.models import load_model
import os
import shutil
import argparse
import autokeras as ak
from tensorflow.keras.models import load_model
import autokeras as ak
import pandas as pd
import numpy as np


In [6]:
trials = 1
epochs = 1
patience = 1

In [9]:


project="Facial-keypoints"
run_id= "1.8"
resume_run = True

MAX_TRIALS=int(trials)
EPOCHS=int(epochs)
PATIENCE=int(patience)

### Data Extraction : extract data and save to attached extenal pvc at location /data ###

train_dir_zip='training.zip'
test_dir_zip='test.zip'

from zipfile import ZipFile
with ZipFile(train_dir_zip,'r') as zipObj:
    zipObj.extractall('data')
    print("Train Archive unzipped")
with ZipFile(test_dir_zip,'r') as zipObj:
    zipObj.extractall('data')
    print("Test Archive unzipped")


## Data preprocess 

train_dir='data/training.csv'
test_dir='data/test.csv'
train=pd.read_csv(train_dir)
test=pd.read_csv(test_dir)

train=train.dropna()
train=train.reset_index(drop=True)

X_train=[]
Y_train=[]

for img in train['Image']:
    X_train.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_train=np.reshape(X_train,(-1,96,96,1))
X_train = np.asarray(X_train).astype('float32')
    
for i in range(len((train))): 
    Y_train.append(np.asarray(train.iloc[i][0:30].to_numpy()))
Y_train = np.asarray(Y_train).astype('float32')

## Data training

reg = ak.ImageRegressor(max_trials=MAX_TRIALS)
reg.fit(X_train, Y_train, validation_split=0.15, epochs=EPOCHS)

# Export trained model to externally attached pvc 
my_model = reg.export_model()
my_model.save('data/model_autokeras', save_format="tf")


Trial 1 Complete [00h 01m 05s]
val_loss: 2114.038818359375

Best val_loss So Far: 2114.038818359375
Total elapsed time: 00h 01m 05s
INFO:tensorflow:Oracle triggered exit
67/67 [==============================] - 67s 902ms/step - loss: 1062.2905 - mean_squared_error: 1062.2905


/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./image_regressor/best_model/assets


/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: data/model_autokeras/assets


In [10]:

### Load model 
loaded_model = load_model("data/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

### Pint model summary
print(loaded_model.summary())

test_dir='data/test.csv'
test=pd.read_csv(test_dir)

X_test=[]
for img in test['Image']:
    X_test.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_test=np.reshape(X_test,(-1,96,96,1))
X_test = np.asarray(X_test).astype('float32')

### predict 
y_pred = loaded_model.predict(X_test)

### Create submission file
y_pred= y_pred.reshape(-1,)
submission = pd.DataFrame({'Location': y_pred})
submission.to_csv('data/submission.csv', index=True , index_label='RowId')


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 96, 96, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 96, 96, 3)    0           cast_to_float32[0][0]            
                                                                 cast_to_float32[0][0]            
                                                                 cast_to_float32[0][0]            
______________________________________________________________________________________________

### Submission file is exported to location

```
/home/jovyan/examples-1/facial-keypoints-detection-kaggle-competition/my_data/data/submission.csv
```